## Modelagem da Otimização

O objetivo do trabalho é encontrar uma coalizão de deputados que:

- seja pequena (menor custo),
- tenha votos suficientes para aprovar uma medida constitucional,
- seja internamente coesa nas redes observadas.

### Variáveis do modelo

Para cada deputado i há uma variável binária x_i:

- x_i = 1 se o deputado i está na coalizão.
- x_i = 0 caso contrário.

### Função objetivo

Cada deputado i recebe um custo c_i. A ideia é que deputados pouco conectados nas redes são mais "caros" de trazer para dentro da coalizão.  
O objetivo do modelo é:

- minimizar a soma, sobre todos os deputados i, de c_i multiplicado por x_i.

Em palavras: queremos a coalizão com o menor custo total que ainda satisfaça as restrições abaixo.

### Restrição de votos mínimos

Cada deputado i contribui com um peso de voto v_i.  
Para aprovar uma PEC, é necessário atingir um certo número mínimo de votos. No modelo usei:

- soma, sobre todos os deputados i, de v_i multiplicado por x_i maior ou igual a 308.

Ou seja, a soma de votos dos deputados escolhidos precisa ser pelo menos 308.

### Coesão da coalizão

A partir das redes de covotação e de coautoria, foi construída uma matriz de pesos w_ij que mede a "intensidade" da relação entre cada par de deputados i e j.  
A construção foi:

- pegar a rede de covotação,
- pegar a rede de coautoria,
- combinar as duas em um único peso w_ij usando um parâmetro alfa, que controla o quanto cada rede pesa no resultado.

A coesão ideal da coalizão seria a soma de w_ij para todos os pares de deputados que estão simultaneamente na coalizão. Em notação verbal:

- "somar w_ij para todos os pares (i,j) em que x_i = 1 e x_j = 1".

Impor uma coesão mínima significaria exigir que essa soma seja maior ou igual a um certo limiar theta.

No entanto, essa expressão "x_i vezes x_j" é quadrática. Para escrever isso como um problema de programação inteira mista com restrições lineares, é preciso introduzir variáveis auxiliares y_ij que valem 1 se o par (i,j) está dentro da coalizão e 0 caso contrário, e relacionar y_ij com x_i e x_j por meio de restrições lineares.  
Isso leva a um modelo muito grande, porque há uma variável y_ij para cada aresta da rede.

Para tornar o problema tratável, foi adotada a seguinte simplificação:

- em vez de usar todas as arestas da rede, filtram-se apenas as arestas com peso combinado mais alto (por exemplo, acima de um determinado percentil da distribuição de w_ij);
- a restrição de coesão mínima passa a considerar somente esses relacionamentos mais fortes.

### Implementação

O modelo foi implementado em Julia usando o pacote JuMP e o solver HiGHS.  
O pipeline é o seguinte:

1. Ler as redes de covotação e coautoria (arquivos GEXF gerados em Python).
2. Construir a matriz de pesos combinados w_ij e o custo c_i de cada deputado.
3. Gerar arquivos CSV com as informações dos nós (deputados) e das arestas relevantes.
4. No notebook em Julia:
   - carregar esses CSV,
   - criar as variáveis x_i (e, na versão com coesão explícita, as variáveis auxiliares y_e ligadas às arestas),
   - adicionar a restrição de votos mínimos,
   - opcionalmente, adicionar a restrição de coesão mínima com um valor de theta,
   - definir a função objetivo de minimização do custo,
   - chamar o solver HiGHS.

O modelo simples, sem a restrição de coesão explícita, resolve praticamente de forma instantânea, porque envolve apenas cerca de 600 variáveis binárias e uma restrição principal.

### Resultados qualitativos

No modelo simples (sem coesão explícita), a solução encontrada é uma coalizão mínima de 308 deputados:

- atinge exatamente os 308 votos necessários,
- tem custo total baixo,
- quando se calcula a coesão dessa coalizão em cima da rede completa, a soma de pesos das relações internas é alta.

Na versão com coesão explícita e filtragem das arestas mais fortes, observou-se que a coalizão mínima já atinge a coesão máxima possível dentro do grafo filtrado. Em outras palavras, dentro das conexões mais intensas, não há espaço para aumentar a coesão sem mexer na própria definição dos dados (por exemplo, relaxando o filtro ou usando outro método de ponderação).

### Limitações e extensões possíveis

- O modelo completo, com todas as arestas e todas as variáveis auxiliares y_ij, fica grande demais para o solver HiGHS resolver dentro de um tempo razoável, por causa da quantidade de variáveis binárias e de restrições.
- A filtragem nas arestas é uma forma pragmática de reduzir a dimensão do problema, focando nos relacionamentos mais fortes, mas isso também reduz a variação possível da coesão.
- Extensões possíveis incluem:
  - testar diferentes limiares de filtragem de arestas,
  - usar outras definições de custo dos deputados,
  - estudar heurísticas ou relaxamentos do modelo completo,
  - analisar a evolução temporal das redes de covotação e coautoria.


In [2]:
using Pkg
using CSV
using DataFrames
using JuMP
using HiGHS

In [3]:
nodes_path = "../data/optimization/nodes.csv"
edges_path = "../data/optimization/edges.csv"

nodes = CSV.read(nodes_path, DataFrame)
edges = CSV.read(edges_path, DataFrame)

println("Nós:   ", nrow(nodes))
println("Arestas:", nrow(edges))
first(nodes, 5), first(edges, 5)

Nós:   606
Arestas:170010


(5×3 DataFrame
 Row │ id      c            v     
     │ Int64   Float64      Int64 
─────┼────────────────────────────
   1 │ 105534  0.0209314        1
   2 │ 107283  0.000217912      1
   3 │ 108338  0.001333         1
   4 │ 109429  0.000653439      1
   5 │ 112437  0.00198247       1, 5×5 DataFrame
 Row │ u       v       w_vote   w_auth   w_comb  
     │ Int64   Int64   Float64  Float64  Float64 
─────┼───────────────────────────────────────────
   1 │ 105534  107283      1.0      0.0    0.035
   2 │ 105534  108338      3.0      0.0    0.105
   3 │ 105534  109429      3.0      0.0    0.105
   4 │ 105534  112437      3.0      0.0    0.105
   5 │ 105534  115746      3.0      0.0    0.105)

In [4]:
# conjunto de deputados
N = collect(nodes.id)  # vetor com os ids

# custos c[i] e pesos de voto v[i]
c = Dict(nodes.id .=> nodes.c)
v = Dict(nodes.id .=> nodes.v)

# pares (u,v) e pesos combinados das arestas
pairs = [(edges.u[i], edges.v[i]) for i in 1:nrow(edges)]
w = Dict(pairs[i] => edges.w_comb[i] for i in 1:length(pairs))

println("Tamanho de N: ", length(N))
println("Tamanho de pairs: ", length(pairs))

Tamanho de N: 606
Tamanho de pairs: 170010


In [5]:
println("Exemplo de custo c[i]:")
for (k, val) in first(collect(c), 5)
    println(k, " => ", val)
end

println("\nExemplo de peso w[(u,v)]:")
for ((u,v), val) in first(collect(w), 5)
    println("(", u, ",", v, ") => ", val)
end

Exemplo de custo c[i]:
204408 => 0.0009435208414851762
160600 => 0.0009167373309955882
74317 => 0.0007170103531154025
204555 => 0.001414147125867371
73531 => 0.0004538667172937441

Exemplo de peso w[(u,v)]:
(73788,73891) => 0.07
(204496,73460) => 0.7000000000000001
(141398,178985) => 3.63
(204351,204451) => 1.61
(178946,204404) => 2.19


In [6]:
Vmin = 257          # número mínimo de votos, outra opcao seria 257 pq é maioria simples
# 308 é o minimo necessário pra aprovar uma PEC
theta = 0       # chute inicial de coesão mínima, ajuste depois

println("Vmin = ", Vmin)
println("theta = ", theta)

Vmin = 257
theta = 0


In [7]:
model = Model(HiGHS.Optimizer)

@variable(model, x[i in N], Bin)

@objective(model, Min, sum(c[i] * x[i] for i in N))

@constraint(model, sum(v[i] * x[i] for i in N) >= Vmin)

optimize!(model)


Running HiGHS 1.12.0 (git hash: 755a8e027): Copyright (c) 2025 HiGHS under MIT licence terms
MIP has 1 row; 606 cols; 606 nonzeros; 606 integer variables (606 binary)
Coefficient ranges:
  Matrix  [1e+00, 1e+00]
  Cost    [2e-04, 1e+06]
  Bound   [1e+00, 1e+00]
  RHS     [3e+02, 3e+02]
Presolving model
1 rows, 606 cols, 606 nonzeros  0s
1 rows, 576 cols, 576 nonzeros  0s
Presolve reductions: rows 1(-0); columns 576(-30); nonzeros 576(-30) 

Solving MIP model with:
   1 row
   576 cols (553 binary, 23 integer, 0 implied int., 0 continuous, 0 domain fixed)
   576 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic;
     I => Shifting; J => Feasibility jump; L => Sub-MIP; P => Empty MIP; R => Randomized rounding;
     S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution; Y => HiGHS solution;
     Z => ZI Round; l => Trivial lower; p => Trivial point; u => Trivial upper; z => Trivial zero

        Nodes      |    B&B Tree     |      

In [8]:
using DataFrames
using CSV

# extrair valores de x
x_val = value.(x)

# montar DataFrame com id e se foi selecionado
solution_df = DataFrame(
    id = N,
    selected = [x_val[i] > 0.5 ? 1 : 0 for i in N]
)

# salvar em CSV
CSV.write("../data/optimization/solution_simple.csv", solution_df)
println("Solução salva em ../data/optimization/solution_simple.csv")


Solução salva em ../data/optimization/solution_simple.csv


In [9]:
### calcular coesão da solução atual (sem variáveis y)

# lista de deputados selecionados
x_val = value.(x)
coalition = [i for i in N if x_val[i] > 0.5]

println("Tamanho da coalizão: ", length(coalition))

# agora somamos pesos das arestas internas da coalizão
cohesion_value = 0.0

# transformar coalition num conjunto para lookup rápido
C = Set(coalition)

for (u, v) in pairs
    if (u in C) && (v in C)
        cohesion_value += w[(u, v)]
    end
end

println("Coesão da coalizão encontrada: ", cohesion_value)


Tamanho da coalizão: 257
Coesão da coalizão encontrada: 200574.87500000012


In [10]:
optimize!(model)

println("Status: ", termination_status(model))
println("Objetivo (custo total): ", objective_value(model))

MIP has 1 row; 606 cols; 606 nonzeros; 606 integer variables (606 binary)
Coefficient ranges:
  Matrix  [1e+00, 1e+00]
  Cost    [2e-04, 1e+06]
  Bound   [1e+00, 1e+00]
  RHS     [3e+02, 3e+02]
Presolving model
1 rows, 606 cols, 606 nonzeros  0s
1 rows, 576 cols, 576 nonzeros  0s
Presolve reductions: rows 1(-0); columns 576(-30); nonzeros 576(-30) 

Solving MIP model with:
   1 row
   576 cols (553 binary, 23 integer, 0 implied int., 0 continuous, 0 domain fixed)
   576 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic;
     I => Shifting; J => Feasibility jump; L => Sub-MIP; P => Empty MIP; R => Randomized rounding;
     S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution; Y => HiGHS solution;
     Z => ZI Round; l => Trivial lower; p => Trivial point; u => Trivial upper; z => Trivial zero

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQu

In [11]:
x_val = value.(x)

coalition = [i for i in N if x_val[i] > 0.5]
println("Tamanho da coalizão: ", length(coalition))
println("Alguns deputados na coalizão: ", first(coalition, min(10, length(coalition))))

Tamanho da coalizão: 257
Alguns deputados na coalizão: [107283, 109429, 116379, 121948, 122158, 122974, 132504, 133439, 133810, 136811]


In [12]:
# votos totais
votes_coalition = sum(v[i] for i in coalition)

# coesão total atingida
y_val = value.(y)
cohesion_value = sum(w[(u,v)] * y_val[(u,v)] for (u,v) in pairs)

println("Votos da coalizão: ", votes_coalition)
println("Coesão da coalizão: ", cohesion_value)

LoadError: UndefVarError: `y` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [40]:
using Tables

coal_df = DataFrame(
    id = N,
    in_coalition = [x_val[i] > 0.5 ? 1 : 0 for i in N],
    cost = [c[i] for i in N],
    vote_weight = [v[i] for i in N],
)

CSV.write("../data/optimization/coalition_solution.csv", coal_df)
println("Solução salva em ../data/optimization/coalition_solution.csv")

Solução salva em ../data/optimization/coalition_solution.csv


In [36]:
import Pkg
Pkg.add("Tables")

   Resolving package versions...
    Updating `C:\Users\Utilisateur\.julia\environments\v1.12\Project.toml`
  [bd369af6] + Tables v1.12.1
    Manifest No packages added to or removed from `C:\Users\Utilisateur\.julia\environments\v1.12\Manifest.toml`
Precompiling packages...
  13712.3 ms  ✓ Clp
  1 dependency successfully precompiled in 16 seconds. 111 already precompiled.


In [14]:
using CSV, DataFrames, JuMP, HiGHS, MathOptInterface, Statistics
const MOI = MathOptInterface

nodes_path = "../data/optimization/nodes.csv"
edges_path = "../data/optimization/edges.csv"

nodes = CSV.read(nodes_path, DataFrame)
edges = CSV.read(edges_path, DataFrame)

# Conjunto de deputados
N = collect(nodes.id)

# custos e pesos de voto
c  = Dict(nodes.id .=> nodes.c)   # custo
v2 = Dict(nodes.id .=> nodes.v)   # pesos de voto PARA ESTE MODELO

# Usar TODAS as arestas inicialmente
edges_coh = deepcopy(edges)

println("Arestas totais antes do filtro: ", nrow(edges_coh))

# calcular o percentil 95 dos pesos combinados
wvals = edges_coh.w_comb
q95 = quantile(wvals, 0.85)

println("Percentil 95 de w_comb = ", q95)

# manter só as arestas com peso combinado acima ou igual a esse limiar
edges_coh = edges_coh[edges_coh.w_comb .>= q95, :]

println("Arestas usadas no modelo com coesão (após filtro 95%): ", nrow(edges_coh))

# Vetor de pares (u,v) e índice inteiro para cada aresta
pairs2 = [(edges_coh.u[i], edges_coh.v[i]) for i in 1:nrow(edges_coh)]
E2 = collect(1:length(pairs2))  # índices das arestas

# Pesos combinados indexados por e (não por (u,v))
w2 = Dict(e => edges_coh.w_comb[e] for e in E2)





Arestas totais antes do filtro: 170010
Percentil 95 de w_comb = 2.875
Arestas usadas no modelo com coesão (após filtro 95%): 25556


Dict{Int64, Float64} with 25556 entries:
  24824 => 4.05
  11950 => 4.925
  1703  => 5.365
  12427 => 3.7
  7685  => 91.175
  18374 => 3.26
  3406  => 6.875
  23970 => 3.155
  1090  => 3.42
  2015  => 3.365
  18139 => 3.47
  17088 => 13.365
  11280 => 4.47
  16805 => 3.7
  3220  => 5.505
  11251 => 8.03
  422   => 3.925
  15370 => 4.68
  25327 => 6.12
  15859 => 4.875
  4030  => 53.98
  8060  => 3.525
  14167 => 3.91
  8660  => 3.065
  18475 => 4.4
  ⋮     => ⋮

In [15]:
cohesion_max_filtered = sum(w2[e] for e in E2)
println("Coesão máxima possível (grafo filtrado): ", cohesion_max_filtered)

theta = 0.7 * cohesion_max_filtered  # ou outra fórmula como expliquei
println("Theta escolhido: ", theta)

Coesão máxima possível (grafo filtrado): 194590.00000000314
Theta escolhido: 136213.00000000218


In [16]:
model2 = Model(HiGHS.Optimizer)

# Variáveis x2[i] para deputados
@variable(model2, x2[i in N], Bin)

# Variáveis y2[e] para arestas (índice inteiro)
@variable(model2, y2[e in E2], Bin)

# Objetivo: minimizar custo total
@objective(model2, Min, sum(c[i] * x2[i] for i in N))

# Restrição de votos mínimos
Vmin = 308  # ou o valor que você já usou
@constraint(model2, votes_min, sum(v2[i] * x2[i] for i in N) >= Vmin)

# Linearização: y2[e] = x2[u] * x2[v]
for e in E2
    u, vtx = pairs2[e]
    @constraint(model2, y2[e] <= x2[u])
    @constraint(model2, y2[e] <= x2[vtx])
    @constraint(model2, y2[e] >= x2[u] + x2[vtx] - 1)
end

# Restrição de coesão mínima
theta = 90_000.0
@constraint(model2, cohesion_min, sum(w2[e] * y2[e] for e in E2) >= theta)




cohesion_min : 99.915 y2[1] + 11.82 y2[2] + 19.330000000000002 y2[3] + 4.19 y2[4] + 58.35 y2[5] + 3.49 y2[6] + 8.31 y2[7] + 5.665 y2[8] + 3.14 y2[9] + 51.455 y2[10] + 105.02 y2[11] + 103.705 y2[12] + 2.875 y2[13] + 3.56 y2[14] + 4.42 y2[15] + 8.135 y2[16] + 13.785 y2[17] + 94.775 y2[18] + 3.91 y2[19] + 111.11 y2[20] + 76.54499999999999 y2[21] + 101.77499999999999 y2[22] + 99.565 y2[23] + 106.95 y2[24] + 5.61 y2[25] + 108.425 y2[26] + 114.425 y2[27] + 3.7 y2[28] + 12.96 y2[29] + 2.91 y2[30] + [[...25496 terms omitted...]] + 4.56 y2[25527] + 5.245 y2[25528] + 3.645 y2[25529] + 2.96 y2[25530] + 3.45 y2[25531] + 5.82 y2[25532] + 4.135 y2[25533] + 6.995000000000001 y2[25534] + 6.33 y2[25535] + 3.365 y2[25536] + 3.9800000000000004 y2[25537] + 3.4000000000000004 y2[25538] + 4.3100000000000005 y2[25539] + 2.9250000000000003 y2[25540] + 3.085 y2[25541] + 3.05 y2[25542] + 3.015 y2[25543] + 22.855 y2[25544] + 3.26 y2[25545] + 3.35 y2[25546] + 3.1850000000000005 y2[25547] + 10.540000000000001 y2[2

In [17]:
set_optimizer_attribute(model2, "time_limit", 180.0)      # 60 segundos
set_optimizer_attribute(model2, "log_to_console", true)  # ver log do HiGHS
set_optimizer_attribute(model2, "mip_rel_gap", 0.01)     # parar com gap de 1%

optimize!(model2)

println("Status: ", termination_status(model2))

if termination_status(model2) in [MOI.OPTIMAL, MOI.TIME_LIMIT, MOI.OTHER_LIMIT]
    obj_val   = objective_value(model2)
    obj_bound = objective_bound(model2)
    println("Melhor solução (upper bound): ", obj_val)
    println("Melhor limite inferior (lower bound): ", obj_bound)
    if obj_val != 0.0
        println("Gap relativo: ", (obj_val - obj_bound) / abs(obj_val))
    end
end


Running HiGHS 1.12.0 (git hash: 755a8e027): Copyright (c) 2025 HiGHS under MIT licence terms
MIP has 76670 rows; 26162 cols; 205054 nonzeros; 26162 integer variables (26162 binary)
Coefficient ranges:
  Matrix  [1e+00, 2e+02]
  Cost    [2e-04, 1e+06]
  Bound   [1e+00, 1e+00]
  RHS     [1e+00, 9e+04]
Presolving model
76670 rows, 26162 cols, 205054 nonzeros  0s
76670 rows, 26150 cols, 205042 nonzeros  0s
Presolve reductions: rows 76670(-0); columns 26150(-12); nonzeros 205042(-12) 

Solving MIP model with:
   76670 rows
   26150 cols (26145 binary, 5 integer, 0 implied int., 0 continuous, 0 domain fixed)
   205042 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic;
     I => Shifting; J => Feasibility jump; L => Sub-MIP; P => Empty MIP; R => Randomized rounding;
     S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution; Y => HiGHS solution;
     Z => ZI Round; l => Trivial lower; p => Trivial point; u => Trivial upper; z => Trivia

In [18]:
if termination_status(model2) in [MOI.OPTIMAL, MOI.TIME_LIMIT, MOI.OTHER_LIMIT]

    x2_val = value.(x2)
    coalition2 = [i for i in N if x2_val[i] > 0.5]

    println("Tamanho da coalizão (modelo com coesão): ", length(coalition2))

    # usar o dicionário v2 (não o v que pode ter sido sobrescrito)
    votes_coalition2 = sum(v2[i] for i in coalition2)
    println("Votos da coalizão (modelo com coesão): ", votes_coalition2)

    # Coesão atingida (somando as arestas em E2)
    C2 = Set(coalition2)
    cohesion_value2 = 0.0
    for e in E2
        u, vtx = pairs2[e]
        if (u in C2) && (vtx in C2)
            cohesion_value2 += w2[e]
        end
    end
    println("Coesão da coalizão (modelo com coesão): ", cohesion_value2)

    println("Custo total (modelo com coesão): ", objective_value(model2))
end



Tamanho da coalizão (modelo com coesão): 308
Votos da coalizão (modelo com coesão): 308
Coesão da coalizão (modelo com coesão): 188472.1850000025
Custo total (modelo com coesão): 0.22467623330736325
